### new reviews with LM 

In [1]:
%%capture
!pip uninstall -y bitsandbytes
!pip install -U bitsandbytes

!pip install datasets
!pip install accelerate
!pip install peft

!pip install -U trl
!pip install --upgrade typing_extensions

In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import pipeline

In [2]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type= 'nf4',
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype=torch.bfloat16
)
model_id = 'beomi/gemma-ko-7b'
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config = bnb_config, device_map={"":0})

config.json:   0%|          | 0.00/668 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/2.93G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/2.92G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/2.99G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/2.92G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/2.37G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

In [4]:
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer
)

Device set to use cuda:0


In [58]:
train = pd.read_csv('./train.csv', encoding = 'utf-8-sig')

In [86]:
def get_random_samples(df, n=10, random_state=None):
    sample_rows = df.sample(n=n, random_state=random_state)

    samples = []
    for index, row in sample_rows.iterrows():
        sample = f"{row['output']} \n"
        samples.append(sample)

    return samples 

In [101]:
def create_example(): 
  samples = get_random_samples(train, n=3)
    
  prompt = (
    "You have to make one korean sentence."
    "The content of the sentence should be about stay at hotel or guest house. \n"
    "Below are examples of korean review senteces: \n"
  )
  prompt = prompt + "".join(samples)

  outputs = pipe(
      prompt,
      do_sample=True,
      temperature=0.2,
      top_p=0.9,
      max_new_tokens=100,
      eos_token_id=pipe.tokenizer.eos_token_id
  )

  generated_text = outputs[0]['generated_text']
  result = generated_text[len(prompt):].strip()

  # clean
  # result = result.split("'")[0]
  result = result.split("\n")[0]

  return result 

In [104]:
result = create_example()
print(result)

부산역에서 가까운 호텔을 찾다가 선택한 곳입니다. 부산역에서 도보로 10분 정도 걸리는 거리에 위치해 있고, 부산역에서 호텔까지 가는 길에 롯데리아, 편의점, 맥도날드 등이 있어서 좋았습니다. 객실은 깔끔하고 넓은 편이었고, 침구도 깨끗했습니다.


In [ ]:
example_reviews = []

for i in range(5000): 
    print(f"index: {i}") 
    result = create_example()
    if result not in example_reviews: 
        example_reviews.append(result)

index: 0
index: 1
index: 2
index: 3
index: 4
index: 5
index: 6
index: 7
index: 8
index: 9
index: 10
index: 11
index: 12
index: 13
index: 14
index: 15
index: 16
index: 17
index: 18
index: 19
index: 20
index: 21
index: 22
index: 23
index: 24
index: 25
index: 26
index: 27
index: 28
index: 29
index: 30
index: 31
index: 32
index: 33
index: 34
index: 35
index: 36
index: 37
index: 38
index: 39
index: 40
index: 41
index: 42
index: 43
index: 44
index: 45
index: 46
index: 47
index: 48
index: 49
index: 50
index: 51
index: 52
index: 53
index: 54
index: 55
index: 56
index: 57
index: 58
index: 59
index: 60
index: 61
index: 62
index: 63
index: 64
index: 65
index: 66
index: 67
index: 68
index: 69
index: 70
index: 71
index: 72
index: 73
index: 74
index: 75
index: 76
index: 77
index: 78
index: 79
index: 80
index: 81
index: 82
index: 83
index: 84
index: 85
index: 86
index: 87
index: 88
index: 89
index: 90
index: 91
index: 92
index: 93
index: 94
index: 95
index: 96
index: 97
index: 98
index: 99
index: 100

In [48]:
example_reviews

['2019년 11월 18일부터 2019년 11월 20일까지 2박 3일 동안 묵었던 사람입니다. ',
 '100% 재방문 의사 없음. ',
 '2019년 12월 26일부터 28일까지 2박 3일간 묵었던 트윈룸입니다. ',
 '2019년 12월 28일 체크아웃 후 2020년 1월 10일까지 피드백이 없네요. ',
 '2019년 10월 14일 체크아웃했는데 아직까지도 아무런 피드백이 없네요. ',
 '10월 26일 체크아웃했는데 11월 10일까지 아무런 피드백이 없네요. ',
 '2019년 12월 26일부터 2020년 1월 1일까지 5박을 했는데, 2019년 12월 26일',
 '2019년 12월 24일 체크인, 25일 체크아웃했는데 2019년 12월 31일까지 피드백이 없네요',
 '10월 15일 체크아웃했는데 10월 29일까지도 아무런 피드백이 없네요. ',
 '2019년 12월 29일 1박 2일 숙박했던 트윈룸입니다. ',
 '2019년 11월 16일 체크아웃했는데, 11월 29일까지 아무런 피드백이 없네요. ']

In [ ]:
data = {"output": example_reviews}
df = pd.DataFrame(data)
df.to_csv("train_new.csv", mode='a', index=False, header=False)